# VSB,FEI - Generative AI Workshop

The aim of the workshop is to get an overview of data analysis and deep learning techniques in the generative artificial intelligence (GenAI) domain.

* We will use [Python](https://www.python.org/), [Huggingface](https://huggingface.co/) and [Tensorflow](https://www.tensorflow.org/).

**The exercise will cover these topics:**
* GenAI tools for image data using Huggingface models
<!-- * LLM usage for text generating with Huggingface API -->
* Vector representation of text data and searching for similar words using vector distance 
* Design of own deep learning model for generating "Harry Potter"-like text using Keras framework from scratch

## Deep learning in Python introduction
* This lecture is focused on using word embedding for searching for similar words and RNN usage for text generation.

* We will use Harry Potter books in this lectures for demonstration of training own model in Keras and generating our own HP-like stories.

![meme01](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_meme_01.jpg?raw=true)

## Import of the TensorFlow
The main version of the TensorFlow (TF) is a in the Version package in the field VERSION Since the TensformFlow 2.0 everything was encapsulaed under the KERAS api.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_distances

tf.version.VERSION

# 🔎 How does the neural network work with text?
* Is is capable to process text directly or does it works just with numbers?
* Can you come up with some very simple way how to encode text to numbers?

# 🔎 What is a word embedding?
* Why do we use it?
* What different propeties will it have compared to some naive approaches?

# Word embedding is a vector
* Do you know what is vector?

# $$\vec{w} = \left(w_1, w_2, ..., w_n\right)$$

# 💡You can imagine embedding vector as an array of numbers, e.g. [0.5,0.3,0.1,-0.3,1.2]

![meme03](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_05_enc_arch.png?raw=true)

# The most famous word embedding is perhaps the Word2Vec

## 💡 There are two approaches for a Word2Vec embedding training

* **Continuous bag-of-words model**: 
    * predicts the middle word based on surrounding context words. 
    * the context consists of a few words before and after the current (middle) word. 
    * this architecture is called a bag-of-words model as the order of words in the context is not important.

* **Continuous skip-gram model**: 
    * predicts words within a certain range before and after the current word in the same sentence. 

![w2v](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_07_skip.png?raw=true)
  
* 💡 Bag-of-words model predicts a word given the neighboring context
* 💡 Skip-gram model predicts the context (or neighbors) of a word, given the word itself
* 💡 The context of a word can be represented through a set of skip-gram pairs of *(target_word, context_word)* where *context_word* appears in the neighboring context of target_word.

## We will demonstrate the approach using single sentence

* The context words for each of the 8 words of this sentence are defined by a window size. 
* The window size determines the span of words on either side of a target_word that can be considered a context word.

![w2v_tab](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_07_tab.png?raw=true)

# 💡 The deep learning model de-facto learns which pairs of words are often appear together in text and which do not
* Can you give some word-pairs examples yourself?

# A nice property of word embedding vectors is that vectors of similar meaning are put close together
* If you compute a distance between two similar words, it will be less than for two unrelated words
* E.g. dog - animal X car - cake

## Let's say that the vector is just 2D
* How does 2D vector look like?
* 🔎 Can you calculate distance between two 2D vectors?
* 🔎 How is the formula called for 2D and how for n-D?

# Ok, enough of theory!
## Let's try it practically with a pre-trained vectors! 🙂
* 🔎 Pre-trained on what!?

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [ ]:
path_to_glove_file = 'glove.6B.50d.txt'

# We will take a look on the file structure now

In [ ]:
with open(path_to_glove_file) as f:
    i = 0
    for line in f:
        print(line)
        i += 1
        if i > 5:
            break

# Let's load the file into a dictionary
* key:value structure -> word:vector

In [ ]:
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

## 💡 This is how the embedding latent vector looks like for the word 'audi' and 'bmw'

In [ ]:
embeddings_index['audi']

In [ ]:
embeddings_index['bmw']

## 💡 The cosine similarity of the car brands should be smaller than of some random words
* Why?

# Cosine vs. Euclidean similarity
* 🔎 What is the difference?
* 🔎 How to compute it?

![meme03](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_meme_tf_02.png?raw=true)

## $$cos(\vec{A},\vec{B}) = \frac{\sum_{i=1}^{n} A_i \cdot B_i}{\sqrt{\sum_{i=1}^{n} A_i^2 \cdot \sum_{i=1}^{n} B_i^2}}$$



# Let's try it out! 🙂

In [ ]:
cosine(embeddings_index['audi'], embeddings_index['bmw'])

In [ ]:
cosine(embeddings_index['audi'], embeddings_index['king'])

# For trying the famous queen -> king example we need to build the embedding matrix

![w2v_meme_03](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_07_meme_03.png?raw=true)

In [ ]:
num_tokens = len(embeddings_index.keys())
embedding_dim = 50
hits = 0
misses = 0
word2id = {k:i for i, (k,v) in enumerate(embeddings_index.items())}
id2word = {v:k for k, v in word2id.items()}

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word2id.items():
    embedding_vector = embeddings_index.get(word)
    embedding_matrix[i] = embedding_vector


## Finding the closest words is pretty easy now
* 🔎 What is the distance for two same words?

In [ ]:
c_w = cosine_distances(embedding_matrix[word2id['man']].reshape(-1, 50), embedding_matrix)

In [ ]:
for x in c_w.argsort().ravel()[1:6]:
    print(id2word[x])

In [ ]:
c_w = cosine_distances(embedding_matrix[word2id['woman']].reshape(-1, 50), embedding_matrix)

In [ ]:
for x in c_w.argsort().ravel()[1:6]:
    print(id2word[x])

## The idea is that using the difference between *man* and *woman* should be simillar as *king* and *queen* thus it should be possible to use the difference for searching for analogies

In [ ]:
dist = embeddings_index['man'] - embeddings_index['woman']

In [ ]:
dist

In [ ]:
summed = embeddings_index['queen'] + dist

In [ ]:
summed

In [ ]:
res = cosine_distances(summed.reshape(-1, 50), embedding_matrix)

In [ ]:
res

# And here we go 🙂

In [ ]:
for x in res.argsort().ravel()[1:6]:
    print(id2word[x])

# Deep learning usage in a text-based generative task
* We will use Harry Potter books in this lectures for generating our own stories.
* 

![meme0_final](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/thats_all.jpg?raw=true)